# Pipelines

## Setup

### Environment

* Base Python3.0
* ml.t3.medium
* 2 vCPU + 4 GiB

### Dependencies

In [2]:
!pip uninstall sagemaker -y
!pip uninstall sckikit-learn -y

Found existing installation: sagemaker 2.157.0
Uninstalling sagemaker-2.157.0:
  Successfully uninstalled sagemaker-2.157.0


In [3]:
!pip install -q sagemaker
!pip install -q -U scikit-learn


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


### Imports

In [4]:
import json
import logging
import pathlib

import sagemaker
import sklearn.model_selection
from sagemaker import Session, get_execution_role, image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow import pipeline, pipeline_context, parameters, steps
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.steps import (
    CreateModelStep,
    ProcessingStep,
    TrainingStep,
    TransformStep,
)
from sagemaker.workflow.step_collections import RegisterModel
from scripts import shared_constants

### Define SageMaker session and role

In [5]:
sagemaker_session = Session()
role = get_execution_role()

In [6]:
print(sagemaker.__version__) # 2.157.0

2.157.0


### Define Constants

In [7]:
SKLEARN_FRAMEWORK_VERSION = "1.0-1"
BASE_JOB_NAME = "birds-200-pipeline"
PIPELINE_NAME = "Birds200Pipeline"
IS_LOCAL_PIPELINE = False
REGION = "us-east-1"
MAP_THRESHOLD = 0.1

## Build Model Pipeline

### Parameters

#### Define processing step parameters

In [8]:
PREPROCESSING_INSTANCE_TYPE = "PreprocessingInstanceType"
PREPROCESSING_INSTANCE_COUNT = "PreprocessingInstanceCount"
preprocessing_instance_type = parameters.ParameterString(name=PREPROCESSING_INSTANCE_TYPE, default_value="ml.m5.large")
preprocessing_instance_count = parameters.ParameterInteger(name=PREPROCESSING_INSTANCE_COUNT, default_value=1)

#### Define training step parameters

In [9]:
TRAINING_INSTANCE_TYPE = "TrainingInstanceType"
training_instance_type = parameters.ParameterString(name=TRAINING_INSTANCE_TYPE, default_value="ml.g4dn.4xlarge")

#### Define model step parameters

In [10]:
MODEL_INSTANCE_TYPE = "ModelInstanceType"
model_instance_type = parameters.ParameterString(name=MODEL_INSTANCE_TYPE, default_value="ml.m5.xlarge")

#### Define transform step parameters

In [11]:
TRANSFORM_INSTANCE_TYPE = "TransformInstanceType"
TRANSFORM_INSTANCE_COUNT = "TransformInstanceCount"
transform_instance_type = parameters.ParameterString(name=TRANSFORM_INSTANCE_TYPE, default_value="ml.m5.large")
transform_instance_count = parameters.ParameterInteger(name=TRANSFORM_INSTANCE_COUNT, default_value=1)

#### Define evaluation step parameters

In [12]:
EVALUATION_INSTANCE_TYPE = "EvaluationInstanceType"
EVALUATION_INSTANCE_COUNT = "EvaluationInstanceCount"
evaluation_instance_type = parameters.ParameterString(name=EVALUATION_INSTANCE_TYPE, default_value="ml.m5.large")
evaluation_instance_count = parameters.ParameterInteger(name=EVALUATION_INSTANCE_COUNT, default_value=1)

#### Define register model step parameters

In [13]:
INFERENCE_INSTANCE_TYPE = "InferenceInstanceType"
inference_instance_type = parameters.ParameterString(name=INFERENCE_INSTANCE_TYPE, default_value="ml.m5.large")

### Helpers

In [14]:
def generate_step_name(step):
    return f"{PIPELINE_NAME}-{step}"

### Pipeline Session

Use LocalPipelineSession for initial development. Then when confident with changes, switch to PipelineSession.

In [15]:
if IS_LOCAL_PIPELINE:
    pipeline_session = sagemaker.workflow.pipeline_context.LocalPipelineSession()
else:
    pipeline_session = sagemaker.workflow.pipeline_context.PipelineSession()

### Workflow Steps

#### Processing Step

##### Define SKLearn processor

In [16]:
preprocessing_sklearn_processor = SKLearnProcessor(
    framework_version=SKLEARN_FRAMEWORK_VERSION,
    instance_type=preprocessing_instance_type,
    instance_count=preprocessing_instance_count,
    base_job_name=BASE_JOB_NAME,
    role=role,
    sagemaker_session=pipeline_session,
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


##### Define Preprocessing Step

In [17]:
preprocessing_step = ProcessingStep(
    name=generate_step_name("Preprocessing"),
    processor=preprocessing_sklearn_processor,
    inputs=[
        ProcessingInput(
            source="./scripts",
            destination=str(shared_constants.INPUT_DIR),
            input_name="scripts",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name=str(output_name), source=str(source))
        for (output_name, source) in [
            (shared_constants.TRAIN_CHANNEL, shared_constants.TRAIN_DIR),
            (shared_constants.VALIDATION_CHANNEL, shared_constants.VALIDATION_DIR),
            (shared_constants.TEST_CHANNEL, shared_constants.TEST_DIR),
            (shared_constants.LABELS_CHANNEL, shared_constants.LABELS_DIR),
        ]
    ],
    code="./scripts/install_packages.py",
)

#### Training Step

I utilize the built-in Docker image and model for object detection. One can also use a custom model within a prebuilt image or deploy a custom image. Here are some helpful resources:
* Learn more about training with Amazon SageMaker: [link](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html)
* Explore an example of using PyTorch for MNIST classification in SageMaker: [link](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/pytorch_mnist/pytorch_mnist.ipynb)
* Understand Docker containers in SageMaker: [link](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html)

##### Get training image

In [18]:
training_image = sagemaker.image_uris.retrieve(region=REGION, framework="object-detection", version="latest")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


##### Define estimator

In [19]:
estimator = sagemaker.estimator.Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type=training_instance_type,
    volume_size=50,
    max_run=int(3600*1.5),
    input_mode="File",
    output_path=shared_constants.S3_OUTPUT_OBJECT_KEY,
    sagemaker_session=pipeline_session,
    base_job_name=BASE_JOB_NAME,
)

For information on object-detection hyperparameters, refer to the documentation at: https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-api-config.html.

##### Set hyperparameters

In [20]:
estimator.set_hyperparameters(
    num_classes=len(shared_constants.CLASS_IDS),
    num_training_samples=shared_constants.NUM_TRAINING_SAMPLES,
)

##### Define estimator inputs

In [22]:
estimator_inputs = {
    "train": TrainingInput(
        s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[shared_constants.TRAIN_CHANNEL].S3Output.S3Uri,
        content_type="application/x-recordio",
    ),
    "validation": TrainingInput(
        s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[shared_constants.VALIDATION_CHANNEL].S3Output.S3Uri,
        content_type="application/x-recordio",
    ),
}

##### Define training step

In [23]:
training_step = TrainingStep(
    name=generate_step_name("Training"),
    estimator=estimator,
    inputs=estimator_inputs,
)

#### Model Step

##### Define model from trained estimator data

In [24]:
model = sagemaker.model.Model(
    image_uri=estimator.training_image_uri(),
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

##### Create model step

In [25]:
model_step = ModelStep(
   name=generate_step_name("Model"),
   step_args=model.create(instance_type=model_instance_type),
)

/usr/local/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:270: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


#### Batch Transform step

##### Define transformer

In [26]:
transformer = Transformer(
    model_name=model_step.properties.ModelName,
    instance_count=transform_instance_count,
    instance_type=transform_instance_type,
    sagemaker_session=pipeline_session,
)

##### Define transform step

In [29]:
transform_step = TransformStep(
    name=generate_step_name("Transform"),
    step_args=transformer.transform(
        data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
            shared_constants.TEST_CHANNEL
        ].S3Output.S3Uri
    ),
)


#### Evaluation Step

##### Create the SKLearn processor

In [30]:
evaluation_sklearn_processor = sagemaker.sklearn.processing.SKLearnProcessor(
    framework_version=SKLEARN_FRAMEWORK_VERSION,
    instance_type=evaluation_instance_type,
    instance_count=evaluation_instance_count,
    base_job_name=BASE_JOB_NAME,
    role=role,
    sagemaker_session=pipeline_session,
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


##### Create the evaluation step (i.e. the processing step for evaluation)

In [31]:
evaluation_report_property_file = PropertyFile(
    name="EvaluationReport", output_name=shared_constants.EVALUATION_CHANNEL, path="evaluation.json"
)

In [33]:
evaluation_step = ProcessingStep(
    name=generate_step_name("Evaluation"),
    processor=evaluation_sklearn_processor,
    inputs=[
        ProcessingInput( # A trick that allows imports modules in scripts
            source="./scripts",
            destination=str(shared_constants.INPUT_DIR),
            input_name="scripts",
        ),
        ProcessingInput(
            source=transform_step.properties.TransformOutput.S3OutputPath,
            destination=str(shared_constants.TRANSFORM_DIR),
        ),
        ProcessingInput(
            source=preprocessing_step.properties.ProcessingOutputConfig.Outputs[shared_constants.LABELS_CHANNEL].S3Output.S3Uri,
            destination=str(shared_constants.LABELS_DIR),
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name=shared_constants.EVALUATION_CHANNEL,
            source=str(shared_constants.EVALUATION_DIR),
        )
    ],
    code='scripts/evaluate.py',
    property_files=[evaluation_report_property_file],
)

#### Register Model Step

In [34]:
register_step = RegisterModel(
    name=generate_step_name("RegisterModel"),
    model=model,
    content_types=["image/jpeg", "image/png"],
    response_types=["application/json"],
    inference_instances=[inference_instance_type],
    transform_instances=[transform_instance_type]
)

#### Create Model Step

In [35]:
create_model_step = CreateModelStep(
    name=generate_step_name("CreateModel"),
    model=model,
)

#### Fail Step

In [36]:
fail_step = FailStep(
    name=generate_step_name("Fail"),
    error_message=f"Execution failured due to mAP score < {MAP_THRESHOLD}",
)

#### Condition Step

In [37]:
cond_map_score = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=evaluation_step,
        property_file=evaluation_report_property_file,
        json_path="map"
    ),
    right=MAP_THRESHOLD,
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [38]:
condition_step = ConditionStep(
    name=generate_step_name("CheckMapScore"),
    conditions=[cond_map_score],
    if_steps=[register_step, create_model_step],
    else_steps=[fail_step],
)

### Create the Pipeline

In [39]:
pipeline = sagemaker.workflow.pipeline.Pipeline(
    name=PIPELINE_NAME,
    parameters=[
        preprocessing_instance_type,
        preprocessing_instance_count,
        training_instance_type,
        model_instance_type,
        transform_instance_type,
        transform_instance_count,
        evaluation_instance_type,
        evaluation_instance_count,
        inference_instance_type,
    ],
    steps=[
        preprocessing_step,
        training_step,
        model_step,
        transform_step,
        evaluation_step,
        condition_step
    ],
    sagemaker_session=pipeline_session,
)

#### Inspect the Pipeline Definition

In [40]:
json.loads(pipeline.definition())

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'PreprocessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'PreprocessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.g4dn.4xlarge'},
  {'Name': 'ModelInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'TransformInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'TransformInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'EvaluationInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'EvaluationInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InferenceInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [

### Build the Pipeline

In [41]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:180797159824:pipeline/birds200pipeline',
 'ResponseMetadata': {'RequestId': '595a35f6-1488-4065-a345-95cc555d1cc8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '595a35f6-1488-4065-a345-95cc555d1cc8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Thu, 18 May 2023 20:49:58 GMT'},
  'RetryAttempts': 0}}

In [42]:
execution = pipeline.start()

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()